In [1]:
"""Trains a ResNet on the CIFAR10 dataset.

ResNet v1
[a] Deep Residual Learning for Image Recognition
https://arxiv.org/pdf/1512.03385.pdf

ResNet v2
[b] Identity Mappings in Deep Residual Networks
https://arxiv.org/pdf/1603.05027.pdf
"""
from __future__ import print_function, division
import keras
from keras.layers import Dense, Add, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D
from keras.layers import MaxPooling2D, AveragePooling2D, ZeroPadding2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from keras.datasets import cifar10
import numpy as np
import os
import sys
import h5py
from datetime import datetime

import pydot
import graphviz
# from IPython.display import SVG

from model_specification import *
from data_generator import *
import learning_parameters

# %matplotlib inline

C:\Users\Landscape\Anaconda3\envs\PythonDLCPU\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# ORIG_PATH = %env PATH
# %env PATH=${ORIG_PATH};C:\Users\Landscape\Anaconda3\Library\bin\graphviz

# Model Instantiation

In [3]:
model_parameters = learning_parameters.get_model_parameters()
file_parameters = learning_parameters.get_file_parameters()
training_parameters = learning_parameters.get_training_parameters()


In [14]:
def instantiate_new_model(model_parameters, file_parameters, training_parameters, learning_rate_schedule):
    model, model_name = resnet_model(input_shape = model_parameters['input_shape'], num_output_classes = model_parameters['num_output_classes'], parameters = model_parameters['model_structure'])

    model.compile(loss = 'categorical_crossentropy',
                  optimizer = Adam(lr=learning_rate_schedule(0)),
                  metrics = ['accuracy'])
    print(model_name)
    print(model.summary())
    # plot_model(model, to_file='ResNetModel.png')
    # SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    return model, model_name

In [5]:
batch_size = training_parameters['batch_size']
datagen = H5DataGenerator(file_parameters['data_file_path'], dim_x = model_parameters['input_shape'][0], dim_y = model_parameters['input_shape'][1], dim_z = model_parameters['input_shape'][2], batch_size = batch_size, shuffle = True, n_classes = model_parameters['num_output_classes'])

In [7]:
if not os.path.isdir(file_parameters['save_dir']):
    os.makedirs(file_parameters['save_dir'])
filepath = os.path.join(file_parameters['save_dir'], file_parameters['model_save_file_name'])

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(learning_parameters.learning_rate_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [15]:
model, model_name = instantiate_new_model(model_parameters, file_parameters, training_parameters, learning_parameters.learning_rate_schedule)

Learning rate:  0.001
ResNet26
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
stage0_Input_CONV (Conv2D)      (None, 112, 112, 32) 4736        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
stage0_Input_BN (BatchNormaliza (None, 112, 112, 32) 128         stage0_Input_CONV[0][0]          
______________________________________________________________________________

In [18]:
print('Training starts')
tick = datetime.now()

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(generator = datagen.generate("training"),
                    steps_per_epoch = int(np.ceil(datagen.get_dataset_size("training") / batch_size)),
                    validation_data = datagen.generate("dev"),
                    validation_steps =int(np.ceil(datagen.get_dataset_size("dev") / batch_size)),
                    epochs = training_parameters['epochs'], 
                    verbose = 1, 
                    workers = 1,
                    callbacks=callbacks)

tock = datetime.now()   
diff_time = tock - tick
print('Training Finished. Time taken:' + str(diff_time.total_seconds()))

Training starts
Learning rate:  0.001
Epoch 1/2
170/170 [==============================] - 506s 3s/step - loss: 1.4050 - acc: 0.7588 - val_loss: 5.0364 - val_acc: 0.3015
Learning rate:  0.001
Epoch 2/2
  6/170 [>.............................] - ETA: 7:40 - loss: 1.4104 - acc: 0.6823

KeyboardInterrupt: 

In [80]:
scores = model.evaluate_generator(generator = datagen.generate("testing"),
                            steps = int(np.ceil(datagen.get_dataset_size("testing") / batch_size)), 
                            workers = 1,
                            use_multiprocessing = False) 
# model.evaluate(x_test, y_test, verbose=1)
print(scores)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

[1.8369186917940776, 0.7213541666666666]
Test loss: 1.8369186917940776
Test accuracy: 0.7213541666666666


In [76]:
predictions = model.predict_generator(generator = datagen.generate_input_in_order("testing"),
                            steps = int(np.ceil(datagen.get_dataset_size("testing") / batch_size)), 
                            workers = 1,
                            verbose = 1,
                            use_multiprocessing = False)

g = datagen.generate_label_in_order("testing")
labels = np.array([], dtype = 'uint8')
for i in range(int(np.ceil(datagen.get_dataset_size("testing") / batch_size))):
    labels = np.append(labels, next(g))


[3 2 2 2 2 2 2 2 2 2 3 2 3 2 2 3 2 2 2 2 3 2 3 2 2 2 2 2 2 2 2 3 2 2 2 2 3
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 7 2 7 7 2 7 7 2 7 2 2 2 2 2 2 2 2 2 3 2 3
 3 2 7 2 7 7 1 1 1 1 2 2 1 1 1 1 7 7 2 2 2 2 8 8 8 2 3 2 2 2 2 2 8 2 8 2 2
 2 2 2 2 8 8 8 8 3 2 2 2 8 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2
 1 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 7 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 7 7 7 7 2 7 7 2 1 8 2 2 2 1 2 7 7 7 2 2 2 7 7 7 2 2 2 2 2 2 2 2 7 7
 2 8 8 8 8 7 2 8 8 7 8 8 8 2 2 2 2 2 2 2 2 2 2 2 2 2 7 7 7 7 7 2 2 2 2 2 2
 2 2 7 2 7 7 2 2 7 7 7 2 2 7 7 7 7 7 2 7 2 7 2 7 7 7 2 7 7 2 7 2 7 2 2 2 7
 2 2 2 2 7 7 2 2 7 2 2 2 7 2 2 2 2 7 2 2 2 7 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2]
[[2.6979239e-04 1.1808320e-01 7.4809349e-01 ... 1.1696829e-01
  3.6975329e-03 3.6829910e-03]
 [3.2215452e-04 1.2188974e-01 8.1397206e-01 ... 4.0069271e-02
  4.8422804e-03 4.6306560e-03]
 [2.3741485e-04 9.5851995e-02 8.526964

In [81]:
predictions_flattened = np.argmax(predictions, axis = 1)
print(predictions_flattened)
print(labels)
match_matrix = predictions_flattened == labels
print(match_matrix)
print('Accuracy:' + str(np.mean( match_matrix )))

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 7 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 7 2 2 2 8 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 7 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 8 7 2 8 2 2 1 7 2 2 2 2 2 2 7 7 2 2 2 7 2 8 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 8 8 8 2 2 2 2 2 2 2
 2 2 7 2 7 8 2 2 2 2 8 2 2 2 7 2 2 2 2 2 2 2 2 2 8 7 2 7 2 2 2 2 2 2 2 2 2
 2 2 2 2 8 2 2 2 2 2 2 2 7 2 2 2 2 2 2 2 2 2 2 2 2 2 2 7 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2]
[3 2 2 2 2 2 2 2 2 2 3 2 3 2 2 3 2 2 2 2 3 2 3 2 2 2 2 2 2 2 2 3 2 2 2 2 3
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 7 2 7 7 2 7 7 2 7 2 2 2 2 2 2 2 2 2 3 2 3
 3 2 7 2 7 7 1 1 1 1 2 2 1 1 1 1 7 7 2 2 2 2 8 8 8 2 3 2 2 2 2 2 8 2 8 2 2

# Unused

In [ ]:
# # Load the CIFAR10 data.
# (x_train, y_train), (x_dev, y_dev) = cifar10.load_data()

# # Input image dimensions.
# input_shape = x_train.shape[1:]

# # Normalize data.
# x_train = x_train.astype('float32') / 255
# x_dev = x_test.astype('float32') / 255

# # If subtract pixel mean is enabled
# if subtract_pixel_mean:
#     x_train_mean = np.mean(x_train, axis=0)
#     x_train -= x_train_mean
#     x_dev -= x_train_mean

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_dev.shape[0], 'dev samples')
# print('y_train shape:', y_train.shape)

# # Convert class vectors to binary class matrices.
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_dev = keras.utils.to_categorical(y_dev, num_classes)



In [ ]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])